In [20]:
import pandas as pd 
import numpy as np 

#Dichiaro il path della cartella del dataset
import os
dataset_folder = "/home/aliquodfahriam/tinyML/CMAPSS/CMAPSS"

In [21]:
#Genero i nomi per le colonne dei sensori
sensors = []
for i in range(1,23): #Nel readme dice che i sensori dovrebbero essere 26, tuttavia dai dati ne emergono soltanto 21, il 22esimo dovrebbe essere pieno di NaN,
                    #Tuttavia mettendo il range da 1 a 22 si crea un warning per cui ci sarebbe una perdita di informazione che non sarebbe desiderabile.  
    sensor = "sensor"+str(i)
    sensors.append(sensor)

print(sensors)

del i, sensor

['sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6', 'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17', 'sensor18', 'sensor19', 'sensor20', 'sensor21', 'sensor22']


In [22]:
train = []
test = []
names_train = ['train1', 'train2', 'train3', 'train4']
names_test = ['test1', 'test2', 'test3', 'test4']

files = os.listdir(dataset_folder)
train_files = [file for file in files if file.startswith('train')]
test_files = [file for file in files if file.startswith('test')]
# print(train_files)
# print(test_files)
train_files.sort()
test_files.sort()
print(train_files)
print(test_files)

colnames = ["unit_number", "time_cycle", "op_setting1", "op_setting2", "op_setting3"]

for sensor in sensors:
    colnames.append(sensor)


for file in train_files: 
    df = pd.read_csv(dataset_folder+"/"+file, delimiter=' ', index_col=False, names=colnames)
    train.append(df)

df = train[0]

print(colnames)

#df = pd.read_csv(dataset_folder+"/"+train_files[3], delimiter=' ', index_col=False, names=colnames)


['train_FD001.txt', 'train_FD002.txt', 'train_FD003.txt', 'train_FD004.txt']
['test_FD001.txt', 'test_FD002.txt', 'test_FD003.txt', 'test_FD004.txt']
['unit_number', 'time_cycle', 'op_setting1', 'op_setting2', 'op_setting3', 'sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6', 'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17', 'sensor18', 'sensor19', 'sensor20', 'sensor21', 'sensor22']


In [23]:
## Proviamo a rimuovere gli NaN dai vari dataframe per capire se c'è qualche valore fuori posto in qualche riga.

for df in train: 
    df = df.fillna(value=0, inplace=True)

df = train[0]

counter = 0
for df in train:
    counter += 1 
    value = (df['sensor22'] != 0).any()
    if value: 
        print("Ci sono valori diversi da zero nel dataframe di posto "+ str(counter))
    else:
        print("Tutti i valori sono zero nel dataframe di posto "+str(counter))

#Comprendo da questo che posso procedere a rimuovere la colonna sensor22 da tutti i dataframe in train: 
for df in train: 
    df.drop('sensor22', axis=1, inplace=True)

df = train[0]

Tutti i valori sono zero nel dataframe di posto 1
Tutti i valori sono zero nel dataframe di posto 2
Tutti i valori sono zero nel dataframe di posto 3
Tutti i valori sono zero nel dataframe di posto 4


###### Sensori con valori costanti
Nel papper in esame viene fatto notare come alcuni sensori riportino valori costanti e quindi siano inutili ai fini del nostro esame. Procediamo a verificare che ciò è vero e capiamo se è corretto eliminarli o meno

I sensori da controllare sono: 
- Numero 0
- Numero 4 
- Numero 5
- Numero 9 
- Numero 15 
- Numero 17 
- Numero 18 


Da una prima osservazione si nota come non sia possibile tracciare un modello comune per tutti i dataset di training e si debba quindi ricorrere a una preparazione dei dati più *raffinata* che tenga conto del numero di valori per ogni colonna contenente valori appartenenti ad ogni sensore. 

In [24]:
# df = train[0]
# df1 = train[1]
# df2 = train[2]
# df3 = train[3]

In [25]:
sensors.remove("sensor22")
unique_values_list = []

for dataframe in train: 
    unique_values_per_df = [] 
    
    for sensor in sensors:
       
        unique_values_frame = dataframe[sensor].unique()
        unique_values_per_df.append(unique_values_frame)
    
    unique_values_list.append(unique_values_per_df)

La variabile *unique_values_list* è composta da 4 righe, in cui ciascuna contine i valori unici di ogni dataframe. Si potrebbe quindi pensare di separare i 4 array per ottenere singolarmente i valori unici di ogni pezzo del dataset. 

Ricordiamo che questo processo è necessario per comprendere se ciò che viene affermato all'interno del paper corrisponde a verità o meno. 

Il passo successivo sarà la normalizzazione dei dataframe tramite un operatore di MinMax Normalization

In [26]:
#Attenzione! Ogni riga corrisponde ai valori unici di un sensore 
#i valori posti ad NaN sono una normale conseguenza del processo che abbiamo seguito
unique_1 = unique_values_list[0]
unique_2 = unique_values_list[1]
unique_3 = unique_values_list[2]
unique_4 = unique_values_list[3]

In [27]:
for i in range(0,21): 
    print(str(i)+":      "+str(unique_4[i]))




0:      [445.   491.19 462.54 449.44 518.67 489.05]
1:      [549.68 606.07 548.95 ... 609.03 641.01 608.76]
2:      [1343.43 1477.61 1343.12 ... 1516.36 1607.72 1381.29]
3:      [1112.93 1237.5  1117.05 ... 1265.2  1426.77 1430.56]
4:      [ 3.91  9.35  7.05  5.48 14.62 10.52]
5:      [ 5.7  13.61  5.69  9.    7.97 21.58  9.01  7.98 15.46 15.47 13.62 21.57
 21.59 13.6   5.71 15.49  9.03 21.61 13.65  8.    5.72  9.02 13.66 21.6
 15.5  13.63 15.48  7.99 13.64  8.01  7.96 13.59 15.45  8.99  5.68 21.56
 13.58 15.44 21.55  7.95 13.57 13.56  8.98  5.67 21.54 13.67]
6:      [137.36 332.1  138.18 ... 563.12 401.8  569.04]
7:      [2211.86 2323.66 2211.92 ... 2387.16 1914.85 2387.36]
8:      [8311.32 8713.6  8306.69 ... 9126.53 8403.64 8391.31]
9:      [1.01 1.07 1.02 0.93 1.3  0.94 1.26 1.25 1.08 1.03 1.31 1.27 1.04 1.09
 1.05 1.32 1.1  0.95 1.06 1.29 1.28]
10:      [41.69 43.94 41.66 41.68 36.48 41.44 46.94 41.6  36.24 43.86 41.45 36.42
 43.92 36.43 43.84 44.94 41.7  43.98 47.14 43.9  47.05 3

Si può notare subito come i pattern che riguardano il primo e il terzo dataset siano collegati al fatto che essi rappresentino una singola condizione delle sei possibili, mentre il secondo e il quarto dataset (essendo misti) risultino più complessi da analizzare e, di conseguenza, non ci permettono di essere sicuri nel rimuovere i sensori che abbiamo trovato. 
Per completezza allego le condizioni possibili: 


There are 6 conditions (or combinations) which the 3 operational settings can take.
- Condition 1: Altitude = 0, Mach Number = 0, TRA = 100
- Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100
- Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100
- Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60
- Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100
- Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100


In [28]:
from matplotlib import pyplot as plt
import seaborn as sns 

In [29]:
def plotSensors(train, numeroDF): 
    df = train[numeroDF]
    plt.figure(figsize=(30,30))
    i = 0
    for sensor in sensors: 
        plt.subplot(10, 3, i+1).set_title(sensor)
        #Il problema sta nel cercare di plottare i dati dei sensori di un unico ciclo
        sensordata = df[df]


In [30]:
# Prima ancora di procedere con la normalizzazione dei dati dei sensori potrebbe essere d'aiuto
# completare il dataset con la RUL per ogni elemento all'interno di ogni dataframe.
#Infine plottare i dati dei sensori per comprenderne meglio la distribuzione
